# <center> **Xử lí dữ liệu VIETNET-BROWSER** </center>

In [1]:
import sqlite3
import pandas as pd

In [2]:
# df = pd.read_csv('data/vietnet_data_v2/VietNet_data.csv')
df = pd.read_excel('data/vietnet_data_v2/VietNet_data.xlsx')

In [3]:
import unicodedata

def remove_vietnamese_tone(text):
    text = unicodedata.normalize('NFD', text)
    text = ''.join([c for c in text if unicodedata.category(c) != 'Mn'])
    # Thay thế các ký tự đặc biệt còn lại
    text = text.replace('đ', 'd').replace('Đ', 'D')
    text = text.replace('ă', 'a').replace('Ă', 'A')
    text = text.replace('â', 'a').replace('Â', 'A')
    text = text.replace('ê', 'e').replace('Ê', 'E')
    text = text.replace('ô', 'o').replace('Ô', 'O')
    text = text.replace('ơ', 'o').replace('Ơ', 'O')
    text = text.replace('ư', 'u').replace('Ư', 'U')
    text = text.replace('á', 'a').replace('à', 'a').replace('ả', 'a').replace('ã', 'a').replace('ạ', 'a')
    text = text.replace('é', 'e').replace('è', 'e').replace('ẻ', 'e').replace('ẽ', 'e').replace('ẹ', 'e')
    text = text.replace('í', 'i').replace('ì', 'i').replace('ỉ', 'i').replace('ĩ', 'i').replace('ị', 'i')
    text = text.replace('ó', 'o').replace('ò', 'o').replace('ỏ', 'o').replace('õ', 'o').replace('ọ', 'o')
    text = text.replace('ú', 'u').replace('ù', 'u').replace('ủ', 'u').replace('ũ', 'u').replace('ụ', 'u')
    text = text.replace('ý', 'y').replace('ỳ', 'y').replace('ỷ', 'y').replace('ỹ', 'y').replace('ỵ', 'y')
    return text

In [4]:
def generate_phrases(text):
    words = text.split()
    n = len(words)
    result = []
    for i in range(n):
        for j in range(i+1, n+1):
            phrase = ' '.join(words[i:j])
            result.append(phrase)
    return result

In [5]:
# Tạo kết nối và cursor
conn = sqlite3.connect('data/vietnet_data_v2/VIETNET.db')
cur = conn.cursor()

# Tạo bảng Search_VietNet
cur.execute('''
CREATE TABLE IF NOT EXISTS VIETNET_DATA (
    synset_id TEXT,
    viet_word TEXT,
    viet_definition TEXT,
    viet_example TEXT,
    is_same BOOLEAN,
    PRIMARY KEY (synset_id, viet_word, viet_definition)
)
''')
# Tạo bảng VIETNET_EXACT_SEARCH
cur.execute('''
CREATE TABLE IF NOT EXISTS VIETNET_EXACT_SEARCH (
    tieng TEXT,
    synset_id TEXT,
    PRIMARY KEY (tieng, synset_id))
''')

# Chèn dữ liệu từ DataFrame vào bảng
for _, row in df.iterrows():
    if row.get('is_same') is True:   # kiểm tra nếu is_same == True
        cur.execute('''
            INSERT OR REPLACE INTO VIETNET_DATA (synset_id, viet_word, viet_definition, viet_example, is_same)
            VALUES (?, ?, ?, ?, ?)
        ''', (
            row['match_id'],
            row['word'],
            row['meaning'],
            row['example'],
            row['is_same']
        ))
        cur.execute('''
            INSERT OR REPLACE INTO VIETNET_EXACT_SEARCH (tieng, synset_id)
            VALUES (?, ?)
        ''', (
            row['word'],
            row['match_id']
        ))

# Tạo bảng VIETNET_FUZZ_SEARCH
cur.execute('''
CREATE TABLE IF NOT EXISTS VIETNET_FUZZ_SEARCH (
    tieng TEXT,
    word TEXT,
    synset_id TEXT,
    PRIMARY KEY (tieng, synset_id))
''')

# Chèn dữ liệu từ DataFrame vào bảng FUZZ_SEARCH
for _, row in df.iterrows():
    tieng_list = generate_phrases(row['word'].lower())
    for tieng in tieng_list:
        cur.execute('''
            INSERT OR REPLACE INTO VIETNET_FUZZ_SEARCH (tieng, word, synset_id)
            VALUES (?, ?, ?)
        ''', (
            tieng,
            row['word'].lower(),
            row['match_id']
        ))
    # Thêm dữ liệu không dấu
    cur.execute('''
        INSERT OR REPLACE INTO VIETNET_FUZZ_SEARCH (tieng, word, synset_id)
        VALUES (?, ?, ?)
    ''', (
        remove_vietnamese_tone(row['word'].lower()),
        row['word'].lower(),
        row['match_id']
    ))


conn.commit()
conn.close()

In [6]:
from rapidfuzz import fuzz

def normalize(text):
    return text.lower().strip()

def fuzzy_search_vietnet_search(user_input, db_path='data/vietnet_data_v2/VIETNET.db', threshold=80):
    # Kết nối và đọc bảng vào DataFrame
    conn = sqlite3.connect(db_path)
    df_search = pd.read_sql_query("SELECT * FROM VIETNET_FUZZ_SEARCH", conn)
    conn.close()
    
    query = normalize(user_input)
    results = []
    for _, row in df_search.iterrows():
        tieng = normalize(row['tieng'])
        score = fuzz.ratio(query, tieng)
        if score >= threshold:
            results.append((row['tieng'], row['synset_id'], score))
    # Sắp xếp theo điểm số giảm dần
    results.sort(key=lambda x: -x[2])
    return results

# Ví dụ sử dụng:
user_input = "da phat"
matches = fuzzy_search_vietnet_search(user_input, threshold=80)
for tieng, synset_id, score in matches:
    print(f"{tieng} ({score}): synset_id={synset_id}")

dao phat (93.33333333333333): synset_id=oewn-08115674-n
dao pha (85.71428571428572): synset_id=oewn-03704925-n
day pha (85.71428571428572): synset_id=oewn-05481998-n
da phan (85.71428571428572): synset_id=oewn-05129822-n
dam pha (85.71428571428572): synset_id=oewn-09351652-n
dia hat (85.71428571428572): synset_id=oewn-03930191-n
dia hat (85.71428571428572): synset_id=oewn-08607925-n
dia hat (85.71428571428572): synset_id=oewn-05823643-n
cua phat (80.0): synset_id=oewn-03880643-n
dao phay (80.0): synset_id=oewn-03045617-n
dao phay (80.0): synset_id=oewn-03158739-n
dan chat (80.0): synset_id=oewn-14858055-n
dau phay (80.0): synset_id=oewn-06825147-n
dau phay (80.0): synset_id=oewn-06855902-n
dai phao (80.0): synset_id=oewn-02953692-n
dai phau (80.0): synset_id=oewn-00686090-n
dan nhat (80.0): synset_id=oewn-04139686-n
dao phap (80.0): synset_id=oewn-06195690-n
dac that (80.0): synset_id=oewn-07307418-n
dau phap (80.0): synset_id=oewn-05913746-n
dia chat (80.0): synset_id=oewn-08674880-n
